In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect


### Connect to local database

In [2]:
# Update input below

rds_connection_string = "postgres:foozfooz@localhost:5432/ETL_db2"
#rds_connection_string = "postgres:postgres@localhost:51661/ETL_db"
#rds_connection_string = "localhost:51661/ETL_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


### Check for tables

In [3]:
engine.table_names()

['gun_violence', 'mass_shootings', 'city_population']

In [4]:
# Load in gun violence / mass shootings tables
mass_shooting = pd.read_excel('Mother Jones - Mass Shootings Database, 1982 - 2019.xlsx')
mass_shooting = mass_shooting[['location', 'fatalities', 'injured', 'date', 'case', 'summary', 'age_of_shooter']].copy()

mass_shooting.head()

,location,fatalities,injured,date,case,summary,age_of_shooter
0,"Dayton, OH",9,27,2019-08-04,Dayton entertainment district shooting,"Connor Betts, 24, died during the attack, foll...",24
1,"El Paso, TX",22,26,2019-08-03,El Paso Walmart mass shooting,"Patrick Crusius, 21, who was apprehended by po...",21
2,"Gilroy, CA",3,12,2019-07-28,Gilroy garlic festival shooting,"Santino William LeGan, 19, fired indiscriminat...",19
3,"Virginia Beach, VA",12,4,2019-05-31,Virginia Beach municipal building shooting,"DeWayne Craddock, 40, a municipal city worker ...",40
4,"Aurora, IL",5,6,2019-02-15,Harry Pratt Co. warehouse shooting,"Gary Martin, 45, went on a rampage inside the ...",45


In [5]:
mass_shooting['city'] = mass_shooting['location'].str.split(', ').str[0]
mass_shooting['state'] = mass_shooting['location'].str.split(', ').str[1]
#mass_shooting= mass_shooting.rename(columns={"city": "City", "state": "State", "case":"case_text"})
mass_shooting= mass_shooting.rename(columns={"case":"case_text"})

mass_shooting.head()

,location,fatalities,injured,date,case_text,summary,age_of_shooter,city,state
0,"Dayton, OH",9,27,2019-08-04,Dayton entertainment district shooting,"Connor Betts, 24, died during the attack, foll...",24,Dayton,OH
1,"El Paso, TX",22,26,2019-08-03,El Paso Walmart mass shooting,"Patrick Crusius, 21, who was apprehended by po...",21,El Paso,TX
2,"Gilroy, CA",3,12,2019-07-28,Gilroy garlic festival shooting,"Santino William LeGan, 19, fired indiscriminat...",19,Gilroy,CA
3,"Virginia Beach, VA",12,4,2019-05-31,Virginia Beach municipal building shooting,"DeWayne Craddock, 40, a municipal city worker ...",40,Virginia Beach,VA
4,"Aurora, IL",5,6,2019-02-15,Harry Pratt Co. warehouse shooting,"Gary Martin, 45, went on a rampage inside the ...",45,Aurora,IL


In [6]:
gun_violence = "gun_violence.csv"
gun_violence_df = pd.read_csv(gun_violence)
gun_violence_df.rename(columns={'state':'State'}, inplace=True)
gun_violence_df = gun_violence_df[['State', 'city_or_county', 'n_killed', 'n_injured', 'date', 'participant_name', 'participant_age']].copy()
gun_violence_df= gun_violence_df.rename(columns={"State":"state", "city_or_county": "city", "n_killed" : "fatalities", "n_injured":"injured", "participant_age": "age_of_shooter"})
gun_violence_df.head()


,state,city,fatalities,injured,date,participant_name,age_of_shooter
0,Pennsylvania,Mckeesport,0,4,2013-01-01,0::Julian Sims,0::20
1,California,Hawthorne,1,3,2013-01-01,0::Bernard Gillis,0::20
2,Ohio,Lorain,1,3,2013-01-01,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,0::25||1::31||2::33||3::34||4::33
3,Colorado,Aurora,4,0,2013-01-05,0::Stacie Philbrook||1::Christopher Ratliffe||...,0::29||1::33||2::56||3::33
4,North Carolina,Greensboro,2,2,2013-01-07,0::Danielle Imani Jameison||1::Maurice Eugene ...,0::18||1::46||2::14||3::47


In [8]:
# load in cities table
cities = pd.read_excel('Resources/US City Populations.xlsx')
#gun_violence = pd.read_csv('Resources/gun-violence-data_01-2013_03-2018.csv')
cities= cities.rename(columns={"State": "state", "City":"city", "Population":"population"})
cities.head()

,state,city,population
0,Alabama,Abernant,7599
1,Alabama,Alabaster,31545
2,Alabama,Albertville,21458
3,Alabama,Alexander City,14849
4,Alabama,Andalusia,9081


In [9]:
# Process cities table to generate per capita information

gun_v = gun_violence_df[['state', 'city']]


gun_v['count'] = gun_v.groupby(['state', 'city'])['state'].transform('size')

#gun_v.rename(columns={'city_or_county':'City'}, inplace=True) 
#gun_v.rename(columns={'state':'State'}, inplace=True)

gun_v2 = pd.merge(gun_v, cities, on=['state', 'city'], how='left')

gun_v2 = gun_v2.drop_duplicates()

gun_v2['shootings_per_cap'] = gun_v2['count']/gun_v2['population']

us_state_abbrev = {
   'Alabama': 'AL',
   'Alaska': 'AK',
   'Arizona': 'AZ',
   'Arkansas': 'AR',
   'California': 'CA',
   'Colorado': 'CO',
   'Connecticut': 'CT',
   'Delaware': 'DE',
   'District of Columbia': 'DC',
   'Florida': 'FL',
   'Georgia': 'GA',
   'Hawaii': 'HI',
   'Idaho': 'ID',
   'Illinois': 'IL',
   'Indiana': 'IN',
   'Iowa': 'IA',
   'Kansas': 'KS',
   'Kentucky': 'KY',
   'Louisiana': 'LA',
   'Maine': 'ME',
   'Maryland': 'MD',
   'Massachusetts': 'MA',
   'Michigan': 'MI',
   'Minnesota': 'MN',
   'Mississippi': 'MS',
   'Missouri': 'MO',
   'Montana': 'MT',
   'Nebraska': 'NE',
   'Nevada': 'NV',
   'New Hampshire': 'NH',
   'New Jersey': 'NJ',
   'New Mexico': 'NM',
   'New York': 'NY',
   'North Carolina': 'NC',
   'North Dakota': 'ND',
   'Northern Mariana Islands':'MP',
   'Ohio': 'OH',
   'Oklahoma': 'OK',
   'Oregon': 'OR',
   'Palau': 'PW',
   'Pennsylvania': 'PA',
   'Puerto Rico': 'PR',
   'Rhode Island': 'RI',
   'South Carolina': 'SC',
   'South Dakota': 'SD',
   'Tennessee': 'TN',
   'Texas': 'TX',
   'Utah': 'UT',
   'Vermont': 'VT',
   'Virgin Islands': 'VI',
   'Virginia': 'VA',
   'Washington': 'WA',
   'West Virginia': 'WV',
   'Wisconsin': 'WI',
   'Wyoming': 'WY',
}
states= pd.DataFrame([us_state_abbrev]).transpose().reset_index()
#states.head()
state_ab= states.rename(columns={"index": "state", 0 : "abbrev"})

gun_v3 = pd.merge(gun_v2, state_ab, on='state', how='left')

gun_v3['population'] = gun_v3['population'].astype('Int64')

#gun_v3= gun_v3.rename(columns={"State": "state", "City":"city", "Population":"population", "shootings_per_cap":"shooting_per_cap"})
gun_v3= gun_v3.rename(columns={"shootings_per_cap":"shooting_per_cap"})

C:\Users\Alex\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
gun_v3.head()

,state,city,count,population,shooting_per_cap,abbrev
0,Pennsylvania,Mckeesport,84,NaN,NaN,PA
1,California,Hawthorne,21,87583,0.000240,CA
2,Ohio,Lorain,113,63776,0.001772,OH
3,Colorado,Aurora,166,353108,0.000470,CO
4,North Carolina,Greensboro,522,282586,0.001847,NC


In [11]:
engine.table_names()


['gun_violence', 'mass_shootings', 'city_population']

### Use pandas to load csv converted Gun Violence DataFrame into database

In [12]:
gun_violence_df.to_sql(name='gun_violence', con=engine, if_exists='append', index=False)


### Use pandas to load csv converted Mass Shooting DataFrame into database

In [13]:
mass_shooting.head()

,location,fatalities,injured,date,case_text,summary,age_of_shooter,city,state
0,"Dayton, OH",9,27,2019-08-04,Dayton entertainment district shooting,"Connor Betts, 24, died during the attack, foll...",24,Dayton,OH
1,"El Paso, TX",22,26,2019-08-03,El Paso Walmart mass shooting,"Patrick Crusius, 21, who was apprehended by po...",21,El Paso,TX
2,"Gilroy, CA",3,12,2019-07-28,Gilroy garlic festival shooting,"Santino William LeGan, 19, fired indiscriminat...",19,Gilroy,CA
3,"Virginia Beach, VA",12,4,2019-05-31,Virginia Beach municipal building shooting,"DeWayne Craddock, 40, a municipal city worker ...",40,Virginia Beach,VA
4,"Aurora, IL",5,6,2019-02-15,Harry Pratt Co. warehouse shooting,"Gary Martin, 45, went on a rampage inside the ...",45,Aurora,IL


In [14]:
mass_shooting.to_sql(name='mass_shootings', con=engine, if_exists='append', index=False)


### Use pandas to load csv converted City Population DataFrame into database

In [15]:
gun_v3.to_sql(name='city_population', con=engine, if_exists='append', index=False)


### Confirm data has been added by querying the FROM ABOVE table
* NOTE: can also check using pgAdmin

In [16]:
pd.read_sql_query('select * from city_population', con=engine).head()


,id,state,abbrev,city,count,population,shooting_per_cap
0,1,Pennsylvania,PA,Mckeesport,84,NaN,NaN
1,2,California,CA,Hawthorne,21,87583.0,0.000240
2,3,Ohio,OH,Lorain,113,63776.0,0.001772
3,4,Colorado,CO,Aurora,166,353108.0,0.000470
4,5,North Carolina,NC,Greensboro,522,282586.0,0.001847


### Confirm data has been added by querying the ALSO FROM ABOVE table

In [17]:
pd.read_sql_query('select * from gun_violence', con=engine).head()


,id,date,state,participant_name,city,fatalities,injured,gun_type,participant_age,age_of_shooter
0,1,2013-01-01,Pennsylvania,0::Julian Sims,Mckeesport,0,4,None,None,0::20
1,2,2013-01-01,California,0::Bernard Gillis,Hawthorne,1,3,None,None,0::20
2,3,2013-01-01,Ohio,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,Lorain,1,3,None,None,0::25||1::31||2::33||3::34||4::33
3,4,2013-01-05,Colorado,0::Stacie Philbrook||1::Christopher Ratliffe||...,Aurora,4,0,None,None,0::29||1::33||2::56||3::33
4,5,2013-01-07,North Carolina,0::Danielle Imani Jameison||1::Maurice Eugene ...,Greensboro,2,2,None,None,0::18||1::46||2::14||3::47


### Confirm data has been added by querying the ALSO FROM ABOVE 2 table

In [18]:
pd.read_sql_query('select * from mass_shootings', con=engine).head()


,id,case_text,city,state,date,summary,location,fatalities,injured,weapon_details,age_of_shooter
0,1,Dayton entertainment district shooting,Dayton,OH,2019-08-04,"Connor Betts, 24, died during the attack, foll...","Dayton, OH",9,27,None,24
1,2,El Paso Walmart mass shooting,El Paso,TX,2019-08-03,"Patrick Crusius, 21, who was apprehended by po...","El Paso, TX",22,26,None,21
2,3,Gilroy garlic festival shooting,Gilroy,CA,2019-07-28,"Santino William LeGan, 19, fired indiscriminat...","Gilroy, CA",3,12,None,19
3,4,Virginia Beach municipal building shooting,Virginia Beach,VA,2019-05-31,"DeWayne Craddock, 40, a municipal city worker ...","Virginia Beach, VA",12,4,None,40
4,5,Harry Pratt Co. warehouse shooting,Aurora,IL,2019-02-15,"Gary Martin, 45, went on a rampage inside the ...","Aurora, IL",5,6,None,45
